<a href="https://colab.research.google.com/github/i-s-a-b-e-l/attempt2/blob/main/GoatedWhartonModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import math

In [ ]:
season = pd.read_csv("NSL_Regular_Season_Data.csv");

In [ ]:
season.head()

In [ ]:
season = season[['game_id', 'HomeTeam', 'AwayTeam', 'HomeScore', 'AwayScore']]

In [ ]:
season['home_result']

In [ ]:
season.head()

In [ ]:
season.loc[season['HomeScore'] > season['AwayScore'], 'home_result'] = 'W'
season.loc[season['HomeScore'] < season['AwayScore'], 'home_result'] = 'L'
season.loc[season['HomeScore'] == season['AwayScore'], 'home_result'] = 'D'

In [ ]:
season.head()

In [ ]:
season['home_result'].value_counts()

In [ ]:
home_table = season[['game_id', 'HomeTeam', 'home_result']]

In [ ]:
season[['away_result']] = None

In [ ]:
season.loc[season['AwayScore'] > season['HomeScore'], 'away_result'] = 'W'
season.loc[season['AwayScore'] < season['HomeScore'], 'away_result'] = 'L'
season.loc[season['AwayScore'] == season['HomeScore'], 'away_result'] = 'D'

In [ ]:
away_table = season[['game_id', 'AwayTeam', 'away_result']]

In [ ]:
away_table.head()

In [ ]:
teams = pd.read_csv("NSL_Metadata.csv")

In [ ]:
teams = teams[['name', 'Team_ID', 'conference']]

In [ ]:
teams.rename(columns = {'name':'team'}, inplace = True)

In [ ]:
teams.head()

In [ ]:
home_table = home_table.merge(teams, how = "left", left_on = "HomeTeam", right_on = "Team_ID")

In [ ]:
home_table.head()

In [ ]:
home_table.drop("HomeTeam", axis = 1, inplace = True)

In [ ]:
home_table.rename(columns = {'home_result' : 'result'}, inplace = True)

In [ ]:
away_table.head()

In [ ]:
away_table = away_table.merge(teams, how = "left", left_on = "AwayTeam", right_on = "Team_ID")
away_table.drop('AwayTeam', axis = 1, inplace = True)
away_table.rename(columns = {'away_result' : 'result'}, inplace = True)

In [ ]:
league_table = pd.concat([home_table, away_table])

In [ ]:
league_table = league_table.groupby('team')['result'].value_counts().unstack().fillna(0)

In [ ]:
league_table['Points'] = 3 * league_table['W'] + league_table['D']

In [ ]:
league_table.sort_values(by = "Points", ascending = False, inplace = True)

In [ ]:
teams.drop(teams.columns[teams.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

teams.head()

,name,city,state,Team_ID,conference,timezone_UTC
0,Albuquerque Scorpions FC,Albuquerque,New Mexico,ALB,Western,-6
1,Anchorage Avalanche FC,Anchorage,Alaska,ANC,Western,-8
2,Augusta Arrows FC,Augusta,Georgia,AUG,Eastern,-4
3,Bakersfield Bandits FC,Bakersfield,California,BAK,Western,-7
4,Boise Thunderhawks FC,Boise,Idaho,BOI,Western,-6


In [ ]:
season.drop(season.columns[season.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)


# Merge df1 and df2 on HomeTeam
merged_df = pd.merge(season, teams, how='left', left_on='HomeTeam', right_on='Team_ID')

# Merge df1 and df2 on AwayTeam
merged_df = pd.merge(merged_df, teams, how='left', left_on='AwayTeam', right_on='Team_ID', suffixes=('_home', '_away'))

# Calculate timezone difference
merged_df['timezone_diff'] = merged_df['timezone_UTC_away'] - merged_df['timezone_UTC_home']
season['timezone_diff'] = merged_df['timezone_diff']
season.head()

,n,game_id,HomeTeam,AwayTeam,HomeScore,AwayScore,Home_xG,Away_xG,Home_shots,Away_shots,Home_corner,Away_corner,Home_PK_Goal,Away_PK_Goal,Home_PK_shots,Away_PK_shots,Home_ToP,timezone_diff
0,1.0,game_2023_1,PRO,MAN,1.0,1.0,1.63,0.34,17.0,8.0,5.0,4.0,0.0,0.0,0.0,0.0,0.53,0.0
1,2.0,game_2023_2,FOR,LEX,1.0,0.0,0.98,0.90,9.0,11.0,5.0,2.0,0.0,0.0,0.0,0.0,0.57,1.0
2,3.0,game_2023_3,SJU,DES,1.0,2.0,1.23,1.11,16.0,9.0,6.0,7.0,0.0,0.0,0.0,0.0,0.47,-1.0
3,4.0,game_2023_4,FAR,ANC,0.0,1.0,2.27,1.08,4.0,14.0,8.0,3.0,0.0,0.0,0.0,0.0,0.51,-3.0
4,5.0,game_2023_5,DOV,BAK,0.0,0.0,1.90,0.56,17.0,6.0,5.0,3.0,0.0,0.0,0.0,0.0,0.53,-3.0


In [ ]:
#Home and road team win probabilities implied by Elo ratings and home court adjustment
def win_probs(home_elo, away_elo, home_court_advantage) :
  h = math.pow(10, home_elo/400)
  r = math.pow(10, away_elo/400)
  a = math.pow(10, home_court_advantage/400)

  denom = r + a*h
  home_prob = a*h / denom
  away_prob = r / denom

  return home_prob, away_prob

#odds the home team will win based on elo ratings and home court advantage
def home_odds_on(home_elo, away_elo, home_court_advantage) :
  h = math.pow(10, home_elo/400)
  r = math.pow(10, away_elo/400)
  a = math.pow(10, home_court_advantage/400)
  return a*h/r

#this function determines the constant used in the elo rating, based on margin of victory and difference in elo ratings
def elo_k(MOV, elo_diff):
  k = 20
  if MOV>0:
      multiplier=(MOV+3)**(0.8)/(7.5+0.006*(elo_diff))
  else:
      multiplier=(-MOV+3)**(0.8)/(7.5+0.006*(-elo_diff))
  return k*multiplier


#updates the home and away teams elo ratings after a game

def update_elo(home_score, away_score, home_elo, away_elo, home_court_advantage) :
  home_prob, away_prob = win_probs(home_elo, away_elo, home_court_advantage)

  if (home_score - away_score > 0) :
    home_win = 1
    away_win = 0
  else :
    home_win = 0
    away_win = 1

  k = elo_k(home_score - away_score, home_elo - away_elo)

  updated_home_elo = home_elo + k * (home_win - home_prob)
  updated_away_elo = away_elo + k * (away_win - away_prob)

  return updated_home_elo, updated_away_elo




In [ ]:
season.head()

,n,game_id,HomeTeam,AwayTeam,HomeScore,AwayScore,Home_xG,Away_xG,Home_shots,Away_shots,Home_corner,Away_corner,Home_PK_Goal,Away_PK_Goal,Home_PK_shots,Away_PK_shots,Home_ToP,timezone_diff
0,1.0,game_2023_1,PRO,MAN,1.0,1.0,1.63,0.34,17.0,8.0,5.0,4.0,0.0,0.0,0.0,0.0,0.53,0.0
1,2.0,game_2023_2,FOR,LEX,1.0,0.0,0.98,0.90,9.0,11.0,5.0,2.0,0.0,0.0,0.0,0.0,0.57,1.0
2,3.0,game_2023_3,SJU,DES,1.0,2.0,1.23,1.11,16.0,9.0,6.0,7.0,0.0,0.0,0.0,0.0,0.47,-1.0
3,4.0,game_2023_4,FAR,ANC,0.0,1.0,2.27,1.08,4.0,14.0,8.0,3.0,0.0,0.0,0.0,0.0,0.51,-3.0
4,5.0,game_2023_5,DOV,BAK,0.0,0.0,1.90,0.56,17.0,6.0,5.0,3.0,0.0,0.0,0.0,0.0,0.53,-3.0


In [ ]:
elo_df = pd.DataFrame(columns=['game_id', 'H_Team', 'A_Team', 'H_Team_Elo_Before', 'A_Team_Elo_Before', 'H_Team_Elo_After', 'A_Team_Elo_After'])
teams_elo_df = pd.DataFrame(columns=['game_id','Team', 'Elo', 'Date', 'Where_Played', 'Season'])

for index, row in season.iterrows():
  game_id = row['game_id']
  h_team, a_team = row['HomeTeam'], row['AwayTeam']
  h_score, a_score = row['HomeScore'], row['AwayScore']

  h_team_elo_before = 1500

  a_team_elo_before = 1500

  h_team_elo_after, a_team_elo_after = update_elo(h_score, a_score, h_team_elo_before, a_team_elo_before, 69)

  new_row = {'game_id': game_id, 'H_Team': h_team, 'A_Team': a_team, 'H_Team_Elo_Before': h_team_elo_before, 'A_Team_Elo_Before': a_team_elo_before, \
                                                                        'H_Team_Elo_After' : h_team_elo_after, 'A_Team_Elo_After': a_team_elo_after}
  teams_row_one = {'game_id': game_id,'Team': h_team, 'Elo': h_team_elo_before, 'Where_Played': 'Home'}
  teams_row_two = {'game_id': game_id,'Team': a_team, 'Elo': a_team_elo_before, 'Where_Played': 'Away'}

  elo_df = elo_df.append(new_row, ignore_index = True)
  teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
  teams_elo_df = teams_elo_df.append(teams_row_two, ignore_index=True)

In [ ]:
elo_df.drop(elo_df.tail(509-476).index,
        inplace = True)

In [ ]:
elo_df.shape

(476, 7)

In [ ]:
season.head()

,n,game_id,HomeTeam,AwayTeam,HomeScore,AwayScore,Home_xG,Away_xG,Home_shots,Away_shots,Home_corner,Away_corner,Home_PK_Goal,Away_PK_Goal,Home_PK_shots,Away_PK_shots,Home_ToP,timezone_diff
0,1.0,game_2023_1,PRO,MAN,1.0,1.0,1.63,0.34,17.0,8.0,5.0,4.0,0.0,0.0,0.0,0.0,0.53,0.0
1,2.0,game_2023_2,FOR,LEX,1.0,0.0,0.98,0.90,9.0,11.0,5.0,2.0,0.0,0.0,0.0,0.0,0.57,1.0
2,3.0,game_2023_3,SJU,DES,1.0,2.0,1.23,1.11,16.0,9.0,6.0,7.0,0.0,0.0,0.0,0.0,0.47,-1.0
3,4.0,game_2023_4,FAR,ANC,0.0,1.0,2.27,1.08,4.0,14.0,8.0,3.0,0.0,0.0,0.0,0.0,0.51,-3.0
4,5.0,game_2023_5,DOV,BAK,0.0,0.0,1.90,0.56,17.0,6.0,5.0,3.0,0.0,0.0,0.0,0.0,0.53,-3.0


In [ ]:
elo_df.head()

,game_id,H_Team,A_Team,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,A_Team_Elo_After
0,game_2023_1,PRO,MAN,1500,1500,1496.159596,1503.840404
1,game_2023_2,FOR,LEX,1500,1500,1503.249585,1496.750415
2,game_2023_3,SJU,DES,1500,1500,1495.165763,1504.834237
3,game_2023_4,FAR,ANC,1500,1500,1495.165763,1504.834237
4,game_2023_5,DOV,BAK,1500,1500,1496.159596,1503.840404


In [ ]:
final_team_stats = pd.merge(season, elo_df, how='left', left_on='game_id', right_on='game_id')

In [ ]:
final_team_stats.head()

,n,game_id,HomeTeam,AwayTeam,HomeScore,AwayScore,Home_xG,Away_xG,Home_shots,Away_shots,...,Home_PK_shots,Away_PK_shots,Home_ToP,timezone_diff,H_Team,A_Team,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,A_Team_Elo_After
0,1.0,game_2023_1,PRO,MAN,1.0,1.0,1.63,0.34,17.0,8.0,...,0.0,0.0,0.53,0.0,PRO,MAN,1500,1500,1496.159596,1503.840404
1,2.0,game_2023_2,FOR,LEX,1.0,0.0,0.98,0.90,9.0,11.0,...,0.0,0.0,0.57,1.0,FOR,LEX,1500,1500,1503.249585,1496.750415
2,3.0,game_2023_3,SJU,DES,1.0,2.0,1.23,1.11,16.0,9.0,...,0.0,0.0,0.47,-1.0,SJU,DES,1500,1500,1495.165763,1504.834237
3,4.0,game_2023_4,FAR,ANC,0.0,1.0,2.27,1.08,4.0,14.0,...,0.0,0.0,0.51,-3.0,FAR,ANC,1500,1500,1495.165763,1504.834237
4,5.0,game_2023_5,DOV,BAK,0.0,0.0,1.90,0.56,17.0,6.0,...,0.0,0.0,0.53,-3.0,DOV,BAK,1500,1500,1496.159596,1503.840404


In [ ]:
final_team_stats = final_team_stats.dropna()

In [ ]:
final_team_stats.drop(columns = ["H_Team" , "A_Team", "Home_ToP"], axis = 1)

,n,game_id,HomeTeam,AwayTeam,HomeScore,AwayScore,Home_xG,Away_xG,Home_shots,Away_shots,...,Away_corner,Home_PK_Goal,Away_PK_Goal,Home_PK_shots,Away_PK_shots,timezone_diff,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,A_Team_Elo_After
0,1.0,game_2023_1,PRO,MAN,1.0,1.0,1.63,0.34,17.0,8.0,...,4.0,0.0,0.0,0.0,0.0,0.0,1500,1500,1496.159596,1503.840404
1,2.0,game_2023_2,FOR,LEX,1.0,0.0,0.98,0.90,9.0,11.0,...,2.0,0.0,0.0,0.0,0.0,1.0,1500,1500,1503.249585,1496.750415
2,3.0,game_2023_3,SJU,DES,1.0,2.0,1.23,1.11,16.0,9.0,...,7.0,0.0,0.0,0.0,0.0,-1.0,1500,1500,1495.165763,1504.834237
3,4.0,game_2023_4,FAR,ANC,0.0,1.0,2.27,1.08,4.0,14.0,...,3.0,0.0,0.0,0.0,0.0,-3.0,1500,1500,1495.165763,1504.834237
4,5.0,game_2023_5,DOV,BAK,0.0,0.0,1.90,0.56,17.0,6.0,...,3.0,0.0,0.0,0.0,0.0,-3.0,1500,1500,1496.159596,1503.840404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,472.0,game_2023_472,SAS,TAC,1.0,2.0,0.71,0.51,19.0,12.0,...,7.0,0.0,0.0,0.0,0.0,-1.0,1500,1500,1495.165763,1504.834237
472,473.0,game_2023_473,DOV,MOB,1.0,1.0,1.70,0.14,19.0,10.0,...,5.0,0.0,0.0,0.0,0.0,-1.0,1500,1500,1496.159596,1503.840404
473,474.0,game_2023_474,BAK,EUG,3.0,2.0,2.68,1.43,17.0,18.0,...,6.0,1.0,0.0,2.0,1.0,0.0,1500,1500,1503.249585,1496.750415
474,475.0,game_2023_475,LRO,LEX,1.0,2.0,2.10,1.30,17.0,6.0,...,5.0,0.0,0.0,0.0,0.0,1.0,1500,1500,1495.165763,1504.834237


In [ ]:

final_team_stats['Label'] = [1 if x > 0 else 0 for x in final_team_stats['HomeScore'] - final_team_stats['AwayScore']]
final_team_stats.drop(columns=['HomeScore', 'AwayScore'], inplace=True)
final_team_stats = final_team_stats.dropna()

In [ ]:
final_team_stats

,n,game_id,HomeTeam,AwayTeam,Home_xG,Away_xG,Home_shots,Away_shots,Home_corner,Away_corner,...,Away_PK_shots,Home_ToP,timezone_diff,H_Team,A_Team,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,A_Team_Elo_After,Label
0,1.0,game_2023_1,PRO,MAN,1.63,0.34,17.0,8.0,5.0,4.0,...,0.0,0.53,0.0,PRO,MAN,1500,1500,1496.159596,1503.840404,0
1,2.0,game_2023_2,FOR,LEX,0.98,0.90,9.0,11.0,5.0,2.0,...,0.0,0.57,1.0,FOR,LEX,1500,1500,1503.249585,1496.750415,1
2,3.0,game_2023_3,SJU,DES,1.23,1.11,16.0,9.0,6.0,7.0,...,0.0,0.47,-1.0,SJU,DES,1500,1500,1495.165763,1504.834237,0
3,4.0,game_2023_4,FAR,ANC,2.27,1.08,4.0,14.0,8.0,3.0,...,0.0,0.51,-3.0,FAR,ANC,1500,1500,1495.165763,1504.834237,0
4,5.0,game_2023_5,DOV,BAK,1.90,0.56,17.0,6.0,5.0,3.0,...,0.0,0.53,-3.0,DOV,BAK,1500,1500,1496.159596,1503.840404,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,472.0,game_2023_472,SAS,TAC,0.71,0.51,19.0,12.0,7.0,7.0,...,0.0,0.42,-1.0,SAS,TAC,1500,1500,1495.165763,1504.834237,0
472,473.0,game_2023_473,DOV,MOB,1.70,0.14,19.0,10.0,1.0,5.0,...,0.0,0.51,-1.0,DOV,MOB,1500,1500,1496.159596,1503.840404,0
473,474.0,game_2023_474,BAK,EUG,2.68,1.43,17.0,18.0,9.0,6.0,...,1.0,0.49,0.0,BAK,EUG,1500,1500,1503.249585,1496.750415,1
474,475.0,game_2023_475,LRO,LEX,2.10,1.30,17.0,6.0,5.0,5.0,...,0.0,0.51,1.0,LRO,LEX,1500,1500,1495.165763,1504.834237,0


In [ ]:
features = final_team_stats.drop(columns = 'Label')
label = final_team_stats['Label']

In [ ]:
features = features.drop(columns = ["Home_ToP", "n", "A_Team_Elo_Before", "H_Team_Elo_Before", "game_id", "HomeTeam", "AwayTeam", "A_Team", "H_Team"])

In [ ]:
features.head()

,Home_xG,Away_xG,Home_shots,Away_shots,Home_corner,Away_corner,Home_PK_Goal,Away_PK_Goal,Home_PK_shots,Away_PK_shots,timezone_diff,H_Team_Elo_After,A_Team_Elo_After
0,1.63,0.34,17.0,8.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,1496.159596,1503.840404
1,0.98,0.90,9.0,11.0,5.0,2.0,0.0,0.0,0.0,0.0,1.0,1503.249585,1496.750415
2,1.23,1.11,16.0,9.0,6.0,7.0,0.0,0.0,0.0,0.0,-1.0,1495.165763,1504.834237
3,2.27,1.08,4.0,14.0,8.0,3.0,0.0,0.0,0.0,0.0,-3.0,1495.165763,1504.834237
4,1.90,0.56,17.0,6.0,5.0,3.0,0.0,0.0,0.0,0.0,-3.0,1496.159596,1503.840404


In [ ]:
label.head()

0    0
1    1
2    0
3    0
4    0
Name: Label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred))

from pprint import pprint
print('Parameters currently in use:\n')
pprint(model.get_params())

1.0
Parameters currently in use:

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}


In [ ]:
log_model = LogisticRegression(max_iter=310, verbose=2, random_state=42)
log_model.fit(x_train, y_train)
y_pred_log = log_model.predict(x_test)
y_test.head()
print(y_pred_log)
print(metrics.accuracy_score(y_test, y_pred_log))

[0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 1 1 1 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0
 0 0 1 1 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 1 0 0]
1.0


In [ ]:
y_test

402    0
438    0
241    1
268    0
23     1
      ..
28     0
192    0
275    1
446    0
470    0
Name: Label, Length: 96, dtype: int64